# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Trusting the compiler”

This lesson is more of a conversation about _type inference_. Isaac makes the following points:

- the C# `var` keyword implements compile-time, local-scope type inference while the `dynamic` keyword makes similar attempts at runtime and has nothing to do with F#
- F# type inference is based on [the Hindley-Milner (HM) type system](https://en.wikipedia.org/wiki/Hindley%E2%80%93Milner_type_system)
- because F# type inference can “escape local scope,” refactoring is made easier
- type inference may not work well with a type from a C# library
- in F#, “implicit conversions” ([type coercion](https://en.wikipedia.org/wiki/Type_conversion#Implicit_type_conversion)) are not allowed
- overloaded F# functions are not allowed:

>You can create (or reference from C# libraries) _classes_ that contain methods that are overloaded, but _functions_ declared using the `let` syntax can’t be overloaded. For this reason, type inference doesn’t completely function on classes and methods.

The following invalid C# dramatizes what F# is capable of:


In [ ]:
public static var CloseAccount(var balance, var customer)
{
    if(balance < 0) return false;
    else
    {
        customer.Withdraw(customer.AccountBalance);
        customer.Close();
        return true;
    }
}

Error: (1,32): error CS0825: The contextual keyword 'var' may only appear within a local variable declaration or in script code
(1,45): error CS0825: The contextual keyword 'var' may only appear within a local variable declaration or in script code
(1,15): error CS0825: The contextual keyword 'var' may only appear within a local variable declaration or in script code

In [ ]:
let closeAccount(balance, customer) =
    if balance < 0 then false
    else
        customer.Withdraw customer.AccountBalance
        customer.Close()
        true

Error: input.fsx (4,9)-(4,26) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (5,9)-(5,23) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.

Even though the F# error message above is much longer, notice how the type `balance` is inferred as `int` (but it probably should be `decimal`). On the other extreme, `customer` cannot be inferred at all and looks like a type that is saturated in OOP complexity:

- `customer` looks like it has a function (method) `Withdraw` that looks like it has one (probably `decimal`) argument
- `customer` looks like it has an `AccountBalance` member (property or field) that is inelegantly being passed to its own function
- `customer` looks like it has a `Close` function that takes no arguments

We can skip ahead in the book (again) to deal with this foreign-looking `customer` (OOP) object and define an F# interface to help the inference engine:


In [ ]:
type ICustomer =
    abstract member AccountBalance : decimal
    abstract member Close : unit -> unit
    abstract member Withdraw : decimal -> unit


We will depend on Isaac to explain to us later how we are defining `ICustomer`. One important detail to mention now is that `unit` in F# is kind of like `void` in C-like languages [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/unit-type)].

We use F# type notation (which resembles Typescript type notation) to explicitly specify the type of `customer`:

In [ ]:
let closeAccount(balance, customer: ICustomer) =
    if balance < 0m then false
    else
        customer.Withdraw customer.AccountBalance
        customer.Close()
        true

Also take note of the `decimal` literal, `0m`, added above. This allows the inference engine to assert that `balance` is `decimal` instead of `int`.

>In the absence of any other information, the F# compiler infers `int` as the type of arithmetic expressions.
>
> [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/symbol-and-operator-reference/arithmetic-operators#operators-and-type-inference)]


## Type-inferred generics

>F# can apply type inference not just on simple values but also for _type arguments_. You can either use an underscore (`_`) to specify a placeholder for the generic type argument, or omit the argument completely.

Here is the underscore approach:


In [ ]:
open System.Collections.Generic

let numbers = List<_>()
numbers.Add(10)
numbers.Add(20)

numbers

index,value
0,10
1,20


Here, the omission approach:


In [ ]:
let numbers = List()
numbers.Add(10)
numbers.Add(20)

numbers

index,value
0,10
1,20


>…F# infers the type based on the first available usage of the type argument.


## Automatic generalization

>F# also automatically makes functions generic when needed.

This inference of the generic type for generics is called _[automatic generalization](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/generics/automatic-generalization)_:


In [ ]:
let createList(first, second) =
    let output = List()

    Console.WriteLine (output.GetType()).FullName

    output.Add(first)
    output.Add(second)

    output

createList(10, 11)

System.Collections.Generic.List`1[[System.Int32, System.Private.CoreLib, Version=5.0.0.0, Culture=neutral, PublicKeyToken=7cec85d7bea7798e]]


index,value
0,10
1,11


We are not changing the type of `List` when we change the type of the arguments:


In [ ]:
createList("ten", "eleven")

System.Collections.Generic.List`1[[System.String, System.Private.CoreLib, Version=5.0.0.0, Culture=neutral, PublicKeyToken=7cec85d7bea7798e]]


index,value
0,ten
1,eleven



## Following the breadcrumbs

The follwing simple type-inference example should _not_ compile for two `typecheck` errors:


In [ ]:
let myFunction(number) =
    if number > "non-number" then "Issac"
    elif number > 20 then "Fred"
    else "Sara"

myFunction 39

Error: input.fsx (3,19)-(3,21) typecheck error This expression was expected to have type
    'string'    
but here has type
    'int'    
input.fsx (6,12)-(6,14) typecheck error This expression was expected to have type
    'string'    
but here has type
    'int'    

The red squiggles here in this notebook are like breadcrumbs, leading back to `number > "non-number"`. Type inference in F# can go seriously wrong when an error like this shows up in a complex chain other functions.

In situations like this, it is important to remember that F# works top down _in order_—so, when `number > "non-number"` is encountered, the inference engine assumes the intention is that `number` must be a `string`.


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.235701+3881a96164de75fca84f5f11027f3606b7878044Build date: 2021-07-11T04:06:39.6100964Zhttps://github.com/dotnet/interactive
